In [1]:
import numpy as np
import math
import glob
import sys

import tensorflow
from tensorflow import float32 as tffloat32
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import newaxis, cast

from sklearn.utils import class_weight

import VenusaurusTransformer
import VenusaurusFileHelper

In [2]:
binWidth_l = 0.5   # Units cm - wire pitch?
targetNBins_l = 50 # This equates to 25cm in length
binWidth_t = 0.5   # Units cm - wire pitch?
targetNBins_t = 20 # This equates to 10cm in length - moliere radius

nVocab_l = 103
embedDim_l = 50

nClasses = 5 # Number of types for classification


nEpochs = 5
batchSize = 128
learningRate = 1e-4

In [20]:

frog = np.array([[1,2,3], [4, 5, 6]])
frog.shape

nbins = np.array([[2], [2]])
nMin = np.array([[0], [0]])
nMax = np.array([[3], [6]])

thisProfile_t, edges_t = np.histogram(frog, 6, range=[nMin, nMax])





ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [3]:
# Here we'll get our information...

# Profiles
longitudinalProfile_start_train = np.empty((0, targetNBins_l))
longitudinalBinIndicies_start_train = np.empty((0, targetNBins_l))
longitudinalProfile_end_train = np.empty((0, targetNBins_l))
longitudinalBinIndicies_end_train = np.empty((0, targetNBins_l))
transverseProfile_train = np.empty((0, targetNBins_t))
transverseBinIndicies_train = np.empty((0, targetNBins_t))

longitudinalProfile_start_test = np.empty((0, targetNBins_l))
longitudinalBinIndicies_start_test = np.empty((0, targetNBins_l))
longitudinalProfile_end_test = np.empty((0, targetNBins_l))
longitudinalBinIndicies_end_test = np.empty((0, targetNBins_l))
transverseProfile_test = np.empty((0, targetNBins_t))
transverseBinIndicies_test = np.empty((0, targetNBins_t))

# Truth
y_train = np.empty((0, nClasses))
y_test = np.empty((0, nClasses))

# Get training file(s)
trainFileNames = glob.glob('/Users/isobel/Desktop/DUNE/Ivysaurus/Venusaurus/files/*/*.npz')
print(trainFileNames)

for trainFileName in trainFileNames :
    print('Reading file: ', str(trainFileName),', This may take a while...')
    
    data = np.load(trainFileName)

    # Profiles
    longitudinalProfile_start_train =  np.concatenate((longitudinalProfile_start_train, data['longitudinalProfile_start_train']), axis=0)
    longitudinalBinIndicies_start_train = np.concatenate((longitudinalBinIndicies_start_train, data['longitudinalBinIndicies_start_train']), axis=0)
    longitudinalProfile_end_train = np.concatenate((longitudinalProfile_end_train, data['longitudinalProfile_end_train']), axis=0)
    longitudinalBinIndicies_end_train = np.concatenate((longitudinalBinIndicies_end_train, data['longitudinalBinIndicies_end_train']), axis=0)
    transverseProfile_train = np.concatenate((transverseProfile_train, data['transverseProfile_train']), axis=0)
    transverseBinIndicies_train = np.concatenate((transverseBinIndicies_train, data['transverseBinIndicies_train']), axis=0)  
                           
    longitudinalProfile_start_test =  np.concatenate((longitudinalProfile_start_test, data['longitudinalProfile_start_test']), axis=0)
    longitudinalBinIndicies_start_test = np.concatenate((longitudinalBinIndicies_start_test, data['longitudinalBinIndicies_start_test']), axis=0)
    longitudinalProfile_end_test = np.concatenate((longitudinalProfile_end_test, data['longitudinalProfile_end_test']), axis=0)
    longitudinalBinIndicies_end_test = np.concatenate((longitudinalBinIndicies_end_test, data['longitudinalBinIndicies_end_test']), axis=0)
    transverseProfile_test = np.concatenate((transverseProfile_test, data['transverseProfile_test']), axis=0)
    transverseBinIndicies_test = np.concatenate((transverseBinIndicies_test, data['transverseBinIndicies_test']), axis=0)                             

    # Truth
    y_train = np.concatenate((y_train, data['y_train']), axis=0)
    y_test = np.concatenate((y_test, data['y_test']), axis=0)

['/Users/isobel/Desktop/DUNE/Ivysaurus/Venusaurus/files/nu/placeholder.npz']
Reading file:  /Users/isobel/Desktop/DUNE/Ivysaurus/Venusaurus/files/nu/placeholder.npz , This may take a while...


In [4]:
# check everything went smoothly
print('longitudinalProfile_start_train: ', longitudinalProfile_start_train.shape)
print('longitudinalBinIndicies_start_train: ', longitudinalBinIndicies_start_train.shape)
print('longitudinalProfile_end_train: ', longitudinalProfile_end_train.shape)
print('longitudinalBinIndicies_end_train: ', longitudinalBinIndicies_end_train.shape)
print('transverseProfile_train: ', transverseProfile_train.shape)
print('transverseBinIndicies_train: ', transverseBinIndicies_train.shape)

print('longitudinalProfile_start_test: ', longitudinalProfile_start_test.shape)
print('longitudinalBinIndicies_start_test: ', longitudinalBinIndicies_start_test.shape)
print('longitudinalProfile_end_test: ', longitudinalProfile_end_test.shape)
print('longitudinalBinIndicies_end_test: ', longitudinalBinIndicies_end_test.shape)
print('transverseProfile_test: ', transverseProfile_test.shape)
print('transverseBinIndicies_test: ', transverseBinIndicies_test.shape)

# Truth
print('y_train: ', y_train.shape)
print('y_test', y_test.shape)

longitudinalProfile_start_train:  (4, 50)
longitudinalBinIndicies_start_train:  (4, 50)
longitudinalProfile_end_train:  (4, 50)
longitudinalBinIndicies_end_train:  (4, 50)
transverseProfile_train:  (4, 20)
transverseBinIndicies_train:  (4, 20)
longitudinalProfile_start_test:  (4, 50)
longitudinalBinIndicies_start_test:  (4, 50)
longitudinalProfile_end_test:  (4, 50)
longitudinalBinIndicies_end_test:  (4, 50)
transverseProfile_test:  (4, 20)
transverseBinIndicies_test:  (4, 20)
y_train:  (4, 5)
y_test (4, 5)


In [5]:
print(longitudinalProfile_start_train[0])

[0.00379395 0.         0.00260592 0.0015677  0.         0.
 0.00602757 0.         0.00176733 0.00361758 0.00401657 0.
 0.00514364 0.00394055 0.0039302  0.00437635 0.00486693 0.
 0.00600197 0.00510372 0.00821073 0.         0.00586229 0.00610512
 0.00677126 0.00524408 0.         0.00420285 0.00410771 0.00408494
 0.00527346 0.         0.00424763 0.00481674 0.0037238  0.00907887
 0.00291653 0.00292758 0.0031234  0.         0.00351914 0.00910257
 0.00268647 0.00228208 0.01459973 0.00832948 0.01542274 0.00893683
 0.0055599  0.        ]


In [6]:
# We need to convert each energy profile bin weight to an index

maxEnergyValue_l = 1
nEnergyBins_l = 1000
energyBinWidth_l = maxEnergyValue_l / float(nEnergyBins_l)

ls_train_mask_above = longitudinalProfile_start_train > maxEnergyValue_l
ls_train_mask_zero = longitudinalProfile_start_train < 0.00001
longitudinalProfile_start_train = np.floor(longitudinalProfile_start_train / energyBinWidth_l)
longitudinalProfile_start_train[ls_train_mask_above] = int(-1)
longitudinalProfile_start_train[ls_train_mask_zero] = int(-2)
longitudinalProfile_start_train = longitudinalProfile_start_train + 2

le_train_mask_above = longitudinalProfile_end_train > maxEnergyValue_l
le_train_mask_zero = longitudinalProfile_end_train < 0.00001
longitudinalProfile_end_train = np.floor(longitudinalProfile_end_train / energyBinWidth_l)
longitudinalProfile_end_train[le_train_mask_above] = int(-1)
longitudinalProfile_end_train[le_train_mask_zero] = int(-2)
longitudinalProfile_end_train = longitudinalProfile_end_train + 2

ls_test_mask_above = longitudinalProfile_start_test > maxEnergyValue_l
ls_test_mask_zero = longitudinalProfile_start_test < 0.00001
longitudinalProfile_start_test = np.floor(longitudinalProfile_start_test / energyBinWidth_l)
longitudinalProfile_start_test[ls_test_mask_above] = int(-1)
longitudinalProfile_start_test[ls_test_mask_zero] = int(-2)
longitudinalProfile_start_test = longitudinalProfile_start_test + 2

le_test_mask_above = longitudinalProfile_end_test > maxEnergyValue_l
le_test_mask_zero = longitudinalProfile_end_test < 0.00001
longitudinalProfile_end_test = np.floor(longitudinalProfile_end_test / energyBinWidth_l)
longitudinalProfile_end_test[le_test_mask_above] = int(-1)
longitudinalProfile_end_test[le_test_mask_zero] = int(-2)
longitudinalProfile_end_test = longitudinalProfile_end_test + 2


In [7]:
print(longitudinalProfile_start_train[0])
print(longitudinalProfile_end_train[0])

[ 5.  0.  4.  3.  0.  0.  8.  0.  3.  5.  6.  0.  7.  5.  5.  6.  6.  0.
  8.  7. 10.  0.  7.  8.  8.  7.  0.  6.  6.  6.  7.  0.  6.  6.  5. 11.
  4.  4.  5.  0.  5. 11.  4.  4. 16. 10. 17. 10.  7.  0.]
[ 3.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  5.
  4.  0.  0.  9. 11.  2.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.]


In [8]:
# Combine
longitudinalProfile_train = np.concatenate((longitudinalProfile_start_train, longitudinalProfile_end_train), axis=1)
longitudinalBinIndicies_train = np.concatenate((longitudinalBinIndicies_start_train, longitudinalBinIndicies_end_train), axis=1)

longitudinalProfile_test = np.concatenate((longitudinalProfile_start_test, longitudinalProfile_end_test), axis=1)
longitudinalBinIndicies_test = np.concatenate((longitudinalBinIndicies_start_test, longitudinalBinIndicies_end_test), axis=1)

In [12]:
# Reshape

longitudinalProfile_train = longitudinalProfile_train.reshape(longitudinalProfile_train.shape[0], (targetNBins_l * 2), 1)
longitudinalBinIndicies_train = longitudinalBinIndicies_train.reshape(longitudinalBinIndicies_train.shape[0], (targetNBins_l * 2), 1)

longitudinalProfile_test = longitudinalProfile_test.reshape(longitudinalProfile_test.shape[0], (targetNBins_l * 2), 1)
longitudinalBinIndicies_test =longitudinalBinIndicies_test.reshape(longitudinalBinIndicies_test.shape[0], (targetNBins_l * 2), 1)

transverseProfile_train = transverseProfile_train.reshape(transverseProfile_train.shape[0], targetNBins_t, 1)
transverseBinIndicies_train = transverseBinIndicies_train.reshape(transverseBinIndicies_train.shape[0], targetNBins_t, 1)

transverseProfile_test = transverseProfile_test.reshape(transverseProfile_test.shape[0], targetNBins_t, 1)
transverseBinIndicies_test = transverseBinIndicies_test.reshape(transverseBinIndicies_test.shape[0], targetNBins_t, 1)

In [13]:
# Check everything went smoothly

# Profiles
print('longitudinalProfile_train: ', longitudinalProfile_train.shape)
print('longitudinalBinIndicies_train: ', longitudinalBinIndicies_train.shape)
print('transverseProfile_train: ', transverseProfile_train.shape)
print('transverseBinIndicies_train: ', transverseBinIndicies_train.shape)

print('longitudinalProfile_test: ', longitudinalProfile_test.shape)
print('longitudinalBinIndicies_test: ', longitudinalBinIndicies_test.shape)
print('transverseProfile_test: ', transverseProfile_test.shape)
print('transverseBinIndicies_test: ', transverseBinIndicies_test.shape)

# Truth
print('y_train: ', y_train.shape)
print('y_test', y_test.shape)

longitudinalProfile_train:  (4, 100, 1)
longitudinalBinIndicies_train:  (4, 100, 1)
transverseProfile_train:  (4, 20, 1)
transverseBinIndicies_train:  (4, 20, 1)
longitudinalProfile_test:  (4, 100, 1)
longitudinalBinIndicies_test:  (4, 100, 1)
transverseProfile_test:  (4, 20, 1)
transverseBinIndicies_test:  (4, 20, 1)
y_train:  (4, 5)
y_test (4, 5)


In [14]:
sequenceLength_l = targetNBins_l * int(2)
nVocab_l = (nEnergyBins_l + 2)

venusaurusModel = VenusaurusTransformer.TransformerModel(sequenceLength_l, nVocab_l, nClasses, embedDim_l)
venusaurusModel.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 positional_embedding (Posi  (None, 100, 50)           50100     
 tionalEmbedding)                                                
                                                                 
 transformer_encoder (Trans  (None, 100, 50)           23832     
 formerEncoder)                                                  
                                                                 
 global_max_pooling1d (Glob  (None, 50)                0         
 alMaxPooling1D)                                                 
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                             

In [15]:
# Define the optimiser and compile the model
optimiser = Adam(learning_rate=learningRate)
venusaurusModel.compile(loss='categorical_crossentropy', optimizer=optimiser, metrics=['accuracy'])

In [16]:
# Create class weights

indexVector = np.argmax(y_test, axis=1)

# muons = 0, protons = 1, pions = 2, electrons = 3, photons = 4, other = 5

nMuons = np.count_nonzero(indexVector == 0)    
nProtons = np.count_nonzero(indexVector == 1)  
nPions = np.count_nonzero(indexVector == 2)  
nElectrons = np.count_nonzero(indexVector == 3)  
nPhotons = np.count_nonzero(indexVector == 4)  

# Normalise to largest
maxParticle = max(nMuons, nProtons, nPions, nElectrons, nPhotons)

#classWeights = {0: maxParticle/nMuons, 1: maxParticle/nProtons, 2: maxParticle/nPions, 3: maxParticle/nElectrons, 4: maxParticle/nPhotons}

classWeights = {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0}


print('Class Weights: ')
print(classWeights)

Class Weights: 
{0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0}


In [17]:
# checkpoint
checkpoint = ModelCheckpoint('/Users/isobel/Desktop/DUNE/Ivysaurus/Venusaurus/files/test', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Reduce the learning rate by a factor of ten when required
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=1e-6, verbose=1)

callbacks_list = [checkpoint, reduce_lr]

history = venusaurusModel.fit(longitudinalProfile_train, y_train, 
    batch_size = batchSize, validation_data=(longitudinalProfile_test, y_test), 
    shuffle=True, epochs=nEpochs, class_weight=classWeights, callbacks=callbacks_list, verbose=2)

Epoch 1/5

Epoch 1: val_accuracy improved from -inf to 0.00000, saving model to /Users/isobel/Desktop/DUNE/Ivysaurus/Venusaurus/files/test
INFO:tensorflow:Assets written to: /Users/isobel/Desktop/DUNE/Ivysaurus/Venusaurus/files/test/assets


INFO:tensorflow:Assets written to: /Users/isobel/Desktop/DUNE/Ivysaurus/Venusaurus/files/test/assets


1/1 - 1s - loss: 4.9098 - accuracy: 0.0000e+00 - val_loss: 2.5444 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04 - 1s/epoch - 1s/step
Epoch 2/5

Epoch 2: val_accuracy did not improve from 0.00000
1/1 - 0s - loss: 3.8962 - accuracy: 0.0000e+00 - val_loss: 2.5142 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04 - 13ms/epoch - 13ms/step
Epoch 3/5

Epoch 3: val_accuracy did not improve from 0.00000
1/1 - 0s - loss: 3.4818 - accuracy: 0.0000e+00 - val_loss: 2.4825 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04 - 13ms/epoch - 13ms/step
Epoch 4/5

Epoch 4: val_accuracy did not improve from 0.00000
1/1 - 0s - loss: 4.0943 - accuracy: 0.0000e+00 - val_loss: 2.4571 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04 - 12ms/epoch - 12ms/step
Epoch 5/5

Epoch 5: val_accuracy did not improve from 0.00000
1/1 - 0s - loss: 3.1935 - accuracy: 0.0000e+00 - val_loss: 2.4272 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04 - 12ms/epoch - 12ms/step
